In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Настройки отображения
pd.set_option('display.max_columns', None)
sns.set(style='whitegrid')


In [ ]:
# Загрузите файлы вручную в Colab (слева -> вкладка Files -> Upload)
orders = pd.read_csv('orders.csv', parse_dates=['created_at', 'put_at', 'closed_at'])
payments = pd.read_csv('payments.csv', parse_dates=['paid_at'])
plan = pd.read_csv('plan.csv', parse_dates=['plan_at'])


In [ ]:
# Сгруппируем фактические платежи по order_id и дате
payments_grouped = payments.groupby(['order_id', 'paid_at'])['paid_sum'].sum().reset_index()

# Рассчитаем накопительный платеж по дате
payments_grouped['paid_cumsum'] = payments_grouped.groupby('order_id')['paid_sum'].cumsum()


In [ ]:
# Объединяем план с фактическими платежами по order_id и дате
plan_sorted = plan.sort_values(['order_id', 'plan_at'])
merged = pd.merge_asof(
    plan_sorted.sort_values(['order_id', 'plan_at']),
    payments_grouped.sort_values(['order_id', 'paid_at']),
    by='order_id',
    left_on='plan_at',
    right_on='paid_at',
    direction='backward'
)

# Заполняем пропуски 0
merged['paid_cumsum'] = merged['paid_cumsum'].fillna(0)

# Рассчитываем просрочку на дату планового платежа
merged['overdue'] = merged['plan_sum_total'] - merged['paid_cumsum']
merged['overdue'] = merged['overdue'].apply(lambda x: max(x, 0))


In [ ]:
# Группируем по дате планового платежа
monthly_overdue = merged.groupby(merged['plan_at'].dt.to_period('M'))['overdue'].sum().reset_index()
monthly_overdue['plan_at'] = monthly_overdue['plan_at'].dt.to_timestamp()

# Строим график
plt.figure(figsize=(12,6))
sns.lineplot(data=monthly_overdue, x='plan_at', y='overdue', marker='o')
plt.title('Динамика общей суммы просрочки по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Сумма просрочки')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### 📊 Вывод

График показывает, как менялась сумма просрочек по плановым платежам во времени. На основе этой динамики можно сделать вывод о том, растёт ли задолженность клиентов или снижается.

Обрати внимание на пики — они могут быть вызваны как реальными ухудшениями поведения клиентов, так и сбоями в платежах, которые стоит проанализировать отдельно.
